In [117]:
def process_steps(steps, initialVariables,variables):
    for step in steps:
        if 'script' in step:
            script_content = step['script']
            expanded_script = expand_variables(script_content, variables, initialVariables)
            print(f"Executing script:\n{expanded_script}\n")
            execute_script(expanded_script)
        elif 'bash' in step:
            bash_command = step['bash']
            expanded_bash_command = expand_variables(bash_command, variables, initialVariables)
            print(f"Executing bash command: {expanded_bash_command}\n")
            update_variable_from_bash(expanded_bash_command, variables)
            execute_bash_command(expanded_bash_command)
        else:
            print("Unsupported step type")

def expand_variables(content, variables, initialVariables):
    for key in variables:
        content = content.replace(f"${{{{ variables.{key} }}}}", str(initialVariables[key]))
        content = content.replace(f"$({key})", str(variables[key]))
    return content

def update_variable_from_bash(bash_command, variables):
    match = re.search(r"\[task\.setvariable variable=(\w+)\](\w+)", bash_command)
    if match:
        variable_name = match.group(1)
        new_value = match.group(2)
        for key in variables:
            if key == variable_name:
                variables[key] = new_value

def execute_script(script):
    print(subprocess.check_output(['bash','-c', script]))

# def execute_bash_command(bash_command):
#     # Use subprocess.run() to execute the bash command
#     print(subprocess.check_output(['bash','-c', bash_command]))

In [118]:
yaml_file_path = 'test.yml'
parsed_data = parse_yaml_script_file(yaml_file_path)

if(parsed_data):
    initialVariables = {}
    for key in parsed_data.get('variables', []):
        initialVariables[key.get('name')] = key.get('value')
    variables = initialVariables.copy()

    steps = parsed_data.get('steps', [])
    process_steps(steps, initialVariables, variables)

Executing script:
echo initialValue # outputs initialValue
echo initialValue


b'initialValue\ninitialValue\n'
Executing bash command: echo "##vso[task.setvariable variable=one]secondValue"

b'##vso[task.setvariable variable=one]secondValue\n'
Executing script:
echo initialValue # outputs initialValue
echo secondValue # outputs secondValue


b'initialValue\nsecondValue\n'


In [100]:
import yaml
import re
import subprocess

In [97]:
def parse_yaml_script_file(file_path):
    with open(file_path) as f:
        try:
            text = f.read()
            parsed_data = yaml.safe_load(text)
            return parsed_data
        except FileNotFoundError:
            print('File not found')
            return None
        except yaml.YAMLError as exc:
            print(exc)
            return None